In [13]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import joblib

In [14]:
# Load tokenizer và model PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert = AutoModel.from_pretrained("vinai/phobert-base")

In [15]:
def extract_features(text, max_length=256):
    """Trích xuất đặc trưng từ PhoBERT"""
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

    # Chạy qua PhoBERT
    with torch.no_grad():
        outputs = phobert(**inputs)

    # Lấy vector đặc trưng (mean pooling)
    embedding = torch.mean(outputs.last_hidden_state, dim=1).squeeze().numpy()

    return embedding


In [16]:
# Đọc dữ liệu từ file CSV
file_path = "vn_news_merged.csv" 
df = pd.read_csv(file_path)

# Kiểm tra cột có trong file
print("Các cột có trong file:", df.columns)

# Xác định cột chứa văn bản và nhãn
text_column = "text"  # Cập nhật đúng tên cột chứa nội dung
label_column = "label"  # Cột chứa nhãn

Các cột có trong file: Index(['text', 'domain', 'label'], dtype='object')


In [17]:
# Kiểm tra dữ liệu rỗng
df = df.dropna(subset=[text_column, label_column])

# Chuyển nhãn thành số nếu cần
if df[label_column].dtype == object:
    df[label_column] = df[label_column].astype("category").cat.codes

# Lấy danh sách văn bản và nhãn
texts = df[text_column].tolist()
labels = df[label_column].tolist()

# Trích xuất đặc trưng từ PhoBERT
X = np.array([extract_features([text]) for text in texts])

In [18]:
# Chia tập dữ liệu 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (359, 768)
X_test shape: (90, 768)


In [19]:
# Huấn luyện mô hình SVM
model = SVC(kernel="linear", probability=True)
model.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [20]:
from sklearn.metrics import accuracy_score

# Dự đoán và đánh giá mô hình
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9778


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Dự đoán nhãn trên tập kiểm tra
y_pred = model.predict(X_test)

# Tính toán các chỉ số đánh giá
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')  
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

# In bảng kết quả đánh giá
results = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1-score"],
    "Score": [accuracy, precision, recall, f1]
})

print(results)

print("\nClassification Report:\n", classification_report(y_test, y_pred))

      Metric     Score
0   Accuracy  0.977778
1  Precision  1.000000
2     Recall  0.957447
3   F1-score  0.978261

Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98        43
           1       1.00      0.96      0.98        47

    accuracy                           0.98        90
   macro avg       0.98      0.98      0.98        90
weighted avg       0.98      0.98      0.98        90



In [22]:
import torch
import joblib
from transformers import AutoTokenizer, AutoModel

# Định nghĩa đường dẫn lưu
svm_model_path = "svm_model.pkl"
tokenizer_path = "phobert_tokenizer"
phobert_model_path = "phobert_model.pt"

# Lưu mô hình SVM
joblib.dump(model, svm_model_path)

# Lưu tokenizer PhoBERT
tokenizer.save_pretrained(tokenizer_path)

# Lưu model PhoBERT
torch.save(phobert.state_dict(), phobert_model_path)


